In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt

log_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/logs/training/job_output-54039124.txt"



In [2]:
trial_pattern = re.compile(r"Trial (\d+): Normalized weights used: \[([^\]]+)\]")
val_loss_pattern = re.compile(r"best_val_loss: ([\deE\.\-]+)")
channel_loss_pattern = re.compile(r"per_channel_val_loss.*\[(.*?)\]")


In [ ]:


trials = []
with open(log_path) as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    trial_match = trial_pattern.search(line)
    if trial_match:
        trial_num = int(trial_match.group(1))
        weights = [float(w) for w in trial_match.group(2).split(",")]
        # best_val_loss ,,per_channel_val_loss in following lines
        best_val_loss = None
        per_channel_val_loss = None
        for j in range(i, min(i+20, len(lines))):
            val_loss_match = val_loss_pattern.search(lines[j])
            if val_loss_match:
                best_val_loss = float(val_loss_match.group(1))
            channel_loss_match = channel_loss_pattern.search(lines[j])
            if channel_loss_match:
                per_channel_val_loss = [float(x) for x in channel_loss_match.group(1).split(",")]
            if best_val_loss is not None and per_channel_val_loss is not None:
                break
        if best_val_loss is not None and per_channel_val_loss is not None:
            trials.append({
                "trial": trial_num,
                "weights": weights,
                "total_loss": best_val_loss,
                "precip_loss": per_channel_val_loss[0],
                "per_channel_val_loss": per_channel_val_loss
            })

df = pd.DataFrame(trials)
print(df.columns)


#Pareto front
plt.figure(figsize=(8,6))
plt.scatter(df["precip_loss"], df["total_loss"], c="blue", label="Trials")
plt.xlabel("Precipitation Channel Validation Loss")
plt.ylabel("Total Validation Loss")
plt.title("Optuna Pareto Front")

# Highlight best (minimum total loss)
best_idx = df["total_loss"].idxmin()
plt.scatter(df.loc[best_idx, "precip_loss"], df.loc[best_idx, "total_loss"], c="red", label="Best Total Loss", s=100, marker="*")

# highlight best precip loss
best_precip_idx = df["precip_loss"].idxmin()
plt.scatter(df.loc[best_precip_idx, "precip_loss"], df.loc[best_precip_idx, "total_loss"], c="gold", label="Best Precip Loss", s=100, marker="*")

plt.legend()
plt.tight_layout()
plt.savefig("optuna_pareto_front.png", dpi=300)
plt.show()

df.to_csv("optuna_trials_table.csv", index=False)
print(df[["trial", "weights", "total_loss", "precip_loss", "per_channel_val_loss"]])

RangeIndex(start=0, stop=0, step=1)


KeyError: 'precip_loss'

<Figure size 800x600 with 0 Axes>